In [1]:
import os
import torch
import random
from torch import nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.models import resnet18
from torchvision.models import ResNet18_Weights
from torchvision import transforms
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
PATH = '/kaggle/input/aaa-ml/avito-auto-moderation'
TRAIN_FILE = 'train_v2.csv'
TEST_FILE = 'sample_submission_v2.csv'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
RANDOM_STATE = 42
BATCH_SIZE = 16

In [3]:
data = pd.read_csv(os.path.join(PATH, TRAIN_FILE))
data.head(5)

,image,label
0,1798.jpg,0
1,372.jpg,1
2,124.jpg,0
3,1155.jpg,0
4,1227.jpg,0


In [4]:
data.isna().sum()

image    0
label    0
dtype: int64

In [5]:
data.groupby('label').count()

,image
label,
0,971
1,172


In [6]:
'''shape_1 = dict()
shape_2 = dict()
for i in data['image']:
    img = read_image(os.path.join(PATH, i))
    shape_1[img.shape[1]] = shape_1.get(img.shape[1], 0) + 1
    shape_2[img.shape[2]] = shape_2.get(img.shape[2], 0) + 1
size_1 = max(shape_1, key=shape_1.get)
size_2 = max(shape_2, key=shape_2.get)'''

"shape_1 = dict()\nshape_2 = dict()\nfor i in data['image']:\n    img = read_image(os.path.join(PATH, i))\n    shape_1[img.shape[1]] = shape_1.get(img.shape[1], 0) + 1\n    shape_2[img.shape[2]] = shape_2.get(img.shape[2], 0) + 1\nsize_1 = max(shape_1, key=shape_1.get)\nsize_2 = max(shape_2, key=shape_2.get)"

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['image'], data['label'], test_size=0.2, random_state=RANDOM_STATE, stratify=data['label'])

In [7]:
class NormalizeImage:
    def __call__(self, image):
        return image.float() / 255.0

MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])

train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    NormalizeImage(),
    transforms.Normalize(mean=MEAN, std=STD),
    v2.GaussianNoise(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
])

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    NormalizeImage(),
    transforms.Normalize(mean=MEAN, std=STD),
])

In [8]:
class ImageDataset(Dataset):
    def __init__(self, images, labels, folder_path, target_len=None, transformation=None):
        self.folder_path = folder_path
        self.images = images
        self.labels = labels
        self.target_len = target_len
        self.transformation = transformation

    def __len__(self):
        if self.target_len:
            return self.target_len
        return len(self.labels)

    def __getitem__(self, idx):
        if self.target_len:
            idx = random.randint(0, len(self.labels) - 1)
        image = read_image(os.path.join(self.folder_path, self.images[idx]))
        if self.transformation:
            image = self.transformation(image)
        label = self.labels[idx]
        return image, label

In [9]:
train_dataset = ImageDataset(X_train.values, y_train.values, PATH, transformation=train_transform)
test_dataset = ImageDataset(X_test.values, y_test.values, PATH, transformation=test_transform)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
class CustomModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = resnet18(weights=ResNet18_Weights.DEFAULT)
        linear = nn.Linear(in_features=self.resnet.fc.in_features, out_features=num_classes, bias=True)
        self.resnet.fc = linear
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.resnet(x)
        x = self.softmax(x)
        return x

In [11]:
model = CustomModel(num_classes=2)
model = model.to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)
weights = torch.tensor([1.0, 5.5]).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(weight=weights)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 181MB/s]


In [12]:
def train_epoch(model, loss_fn, optimizer, dataloader):
    model.train()
    sum_loss = 0
    for i, (x, y) in enumerate(dataloader):
        x = x.to(DEVICE)
        y = y.to(DEVICE)

        preds = model(x)
        loss = loss_fn(preds, y)
        sum_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return sum_loss / len(dataloader)

In [13]:
def eval_epoch(model, loss_fn, dataloader):
    model.eval()
    sum_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for i, (x, y) in enumerate(dataloader):
            x = x.to(DEVICE)
            y = y.to(DEVICE)

            preds = model(x)
            loss = loss_fn(preds, y)
            sum_loss += loss.item()

            y_true = y_true + y.tolist()
            y_pred = y_pred + preds[:, 1].tolist()
    return sum_loss / len(dataloader), roc_auc_score(y_true, y_pred)

In [14]:
epochs = 30
for i in range(epochs):
    train_loss = train_epoch(model, loss_fn, optimizer, train_dataloader)
    test_loss, roc_auc = eval_epoch(model, loss_fn, test_dataloader)

    print(f'Epoch: {i}')
    print(f'Train loss: {train_loss}')
    print(f'Test loss: {test_loss}')
    print(f'ROC AUC: {roc_auc}')
    print('-----------------------------')

Epoch: 0
Train loss: 0.6414627278673237
Test loss: 0.7374277949333191
ROC AUC: 0.5337858220211161
-----------------------------
Epoch: 1
Train loss: 0.5630690049508522
Test loss: 0.682299796740214
ROC AUC: 0.5840120663650076
-----------------------------
Epoch: 2
Train loss: 0.5209482255680807
Test loss: 0.7002211610476176
ROC AUC: 0.6351432880844646
-----------------------------
Epoch: 3
Train loss: 0.49295661110302497
Test loss: 0.6585925777753194
ROC AUC: 0.6135746606334842
-----------------------------
Epoch: 4
Train loss: 0.480744157885683
Test loss: 0.6592238108317058
ROC AUC: 0.6313725490196078
-----------------------------
Epoch: 5
Train loss: 0.44663636797461015
Test loss: 0.7038535912831624
ROC AUC: 0.6143288084464554
-----------------------------
Epoch: 6
Train loss: 0.4565183900553605
Test loss: 0.6693888187408448
ROC AUC: 0.6235294117647059
-----------------------------
Epoch: 7
Train loss: 0.4381017618138215
Test loss: 0.6912757734457652
ROC AUC: 0.6054298642533937
------

KeyboardInterrupt: 

# Submit

In [15]:
sub_data = pd.read_csv(os.path.join(PATH, TEST_FILE))
sub_data.head(5)

,image,score
0,474.jpg,0.5
1,1052.jpg,0.5
2,63.jpg,0.5
3,1713.jpg,0.5
4,116.jpg,0.5


In [16]:
scores = []
model.eval()
with torch.no_grad():
    for i in sub_data['image']:
        image = read_image(os.path.join(PATH, i))
        image = test_transform(image)
        image = image.reshape((1, 3, 256, 256))
        image = image.to(DEVICE)
        
        pred = model(image)
        scores.append(float(pred[0][1]))
sub_data['score'] = scores
sub_data.head(5)

,image,score
0,474.jpg,0.730910
1,1052.jpg,0.679817
2,63.jpg,0.495740
3,1713.jpg,0.542224
4,116.jpg,0.593807


In [17]:
output_dir = '/kaggle/working/'
file_name = 'submission.csv'
sub_data.to_csv(os.path.join(output_dir, file_name), index=False)